In [2]:
from __future__ import annotations

import ast
import typing as T
import collections.abc as C

import numpy as np
import pandas as pd
import altair as alt
from prophet import Prophet
from prophet.plot import add_changepoints_to_plot

In [56]:
def load_dataset_by_id(filepath: str) -> pd.DataFrame:
    def parse_values(x: pd.Series) -> pd.Series:
        keys = ast.literal_eval(x.loc["keys"])[0]
        values = (
            float(x) if x != "null" else pd.NA
            for x in ast.literal_eval(x.loc["values"])[0]
        )
        return pd.Series(dict(zip(keys, values)))

    df = pd.read_csv(filepath, parse_dates=[2], index_col=0)
    values = df.apply(parse_values, axis=1)
    df = pd.concat([df[["configuration_item_id"]], values], axis=1)
    return df.convert_dtypes()

,configuration_item_id,meteo_layer_type,meteo_cloudiness,meteo_wind_velocity,meteo_humidity,meteo_t_underroad,meteo_freezing_point,meteo_wind_direction,meteo_dew_point,meteo_t_road,meteo_wind_gusts,meteo_t_air,meteo_air_pressure
id,,,,,,,,,,,,,
1684,30928,1.0,3,2.3,46.7,41.4,0.0,132,16.9,35.1,4.2,29.5,733
862,30928,1.0,2,2.5,29.4,32.1,0.0,351,3.3,30.3,4.4,22.0,737
1992,24445,1.0,2,0.5,70.8,21.8,0.0,121,-4.7,18.6,1.1,0.0,754
889,30928,1.0,3,2.5,61.6,25.7,0.0,63,11.2,20.7,4.2,18.7,736
1362,30928,1.0,4,2.8,61.5,27.5,0.0,185,14.9,22.2,6.0,22.7,733
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,30928,1.0,2,2.1,58.7,39.1,0.0,82,18.6,33.3,4.2,27.4,736
1095,30928,1.0,3,0.5,60.7,27.0,0.0,205,15.4,24.0,1.9,23.4,736
1130,30928,1.0,3,1.2,90.1,26.0,0.0,231,17.3,26.0,2.4,18.9,734


In [3]:
def load_dataset(filepath: str) -> pd.DataFrame:
    def parse_values(x: pd.Series) -> pd.Series:
        keys = ast.literal_eval(x.loc["keys"])[0]
        values = (
            float(x) if x != "null" else pd.NA
            for x in ast.literal_eval(x.loc["values"])[0]
        )
        return pd.Series(dict(zip(keys, values)))

    df = pd.read_csv(filepath, parse_dates=[2], index_col=2).sort_index()
    values = df.apply(parse_values, axis=1)
    df = pd.concat([df[["id", "configuration_item_id"]], values], axis=1)
    return df.convert_dtypes()

In [4]:
def fit_predict(df: pd.DataFrame):
    model = Prophet().fit(df)
    forecast = model.predict(df)
    forecast["fact"] = df["y"].reset_index(drop=True)
    return forecast

In [5]:
def detect_anomalies(forecast: pd.DataFrame):
    forecasted = forecast[
        ["ds", "trend", "yhat", "yhat_lower", "yhat_upper", "fact"]
    ].copy()

    forecasted["anomaly"] = 0
    forecasted.loc[forecasted["fact"] > forecasted["yhat_upper"], "anomaly"] = 1
    forecasted.loc[forecasted["fact"] < forecasted["yhat_lower"], "anomaly"] = -1

    forecasted["importance"] = 0.0
    forecasted.loc[forecasted["anomaly"] == 1, "importance"] = (
        forecasted["fact"] - forecasted["yhat_upper"]
    ) / forecasted["fact"]
    forecasted.loc[forecasted["anomaly"] == -1, "importance"] = (
        forecasted["yhat_lower"] - forecasted["fact"]
    ) / forecasted["fact"]

    return forecasted

In [33]:
def plot_anomalies(forecasted: pd.DataFrame, title:str|None=None):
    interval = (
        alt.Chart(forecasted)
        .mark_area(interpolate="basis", color="#7FC97F")
        .encode(
            x=alt.X("ds:T", title="date"),
            y="yhat_upper",
            y2="yhat_lower",
            tooltip=["ds", "fact", "yhat_lower", "yhat_upper"],
        )
        .interactive()
        .properties(title=f"Anomalies ({title})")
    )

    fact = (
        alt.Chart(forecasted[forecasted["anomaly"] == 0])
        .mark_circle(size=15, opacity=0.7, color="Black")
        .encode(
            x="ds:T",
            y=alt.Y("fact", title="value"),
            tooltip=["ds", "fact", "yhat_lower", "yhat_upper"],
        )
        .interactive()
    )

    anomalies = (
        alt.Chart(forecasted[forecasted["anomaly"] != 0])
        .mark_circle(size=30, color="Red")
        .encode(
            x="ds:T",
            y=alt.Y("fact", title="Sales"),
            tooltip=["ds", "fact", "yhat_lower", "yhat_upper"],
            size=alt.Size("importance", legend=None),
        )
        .interactive()
    )

    return alt.layer(interval, fact, anomalies).properties(width=870, height=450)

In [14]:
def prepare_col(series: pd.Series) -> pd.DataFrame:
    return pd.DataFrame(
        {
            "ds": series.index,
            "y": series.values,
        }
    )

In [35]:
def predict_and_plot(src: pd.DataFrame, col: str):
    df = prepare_col(src[col])
    
    m = Prophet().fit(df)
    forecast = m.predict(df)
    forecast["fact"] = df["y"].reset_index(drop=True)

    pred = detect_anomalies(forecast)
    return plot_anomalies(pred, col)

In [17]:
X_train = load_dataset("data/train999.csv")
df1 = next(iter(d for s, d in X_train.groupby("configuration_item_id")))
df1.columns

Index(['id', 'configuration_item_id', 'meteo_layer_type', 'meteo_cloudiness',
       'meteo_wind_velocity', 'meteo_humidity', 'meteo_t_underroad',
       'meteo_freezing_point', 'meteo_wind_direction', 'meteo_dew_point',
       'meteo_t_road', 'meteo_wind_gusts', 'meteo_t_air',
       'meteo_air_pressure'],
      dtype='object')

In [38]:
charts = []
for col in df1.columns:
    if col.startswith("meteo"):
        charts.append(predict_and_plot(df1, col))
    
alt.vconcat(*charts).configure_title(fontSize=20)

17:03:22 - cmdstanpy - INFO - Chain [1] start processing
17:03:22 - cmdstanpy - INFO - Chain [1] done processing
17:03:23 - cmdstanpy - INFO - Chain [1] start processing
17:03:23 - cmdstanpy - INFO - Chain [1] done processing
17:03:23 - cmdstanpy - INFO - Chain [1] start processing
17:03:23 - cmdstanpy - INFO - Chain [1] done processing
17:03:23 - cmdstanpy - INFO - Chain [1] start processing
17:03:23 - cmdstanpy - INFO - Chain [1] done processing
17:03:23 - cmdstanpy - INFO - Chain [1] start processing
17:03:23 - cmdstanpy - INFO - Chain [1] done processing
17:03:23 - cmdstanpy - INFO - Chain [1] start processing
17:03:23 - cmdstanpy - INFO - Chain [1] done processing
17:03:24 - cmdstanpy - INFO - Chain [1] start processing
17:03:24 - cmdstanpy - INFO - Chain [1] done processing
17:03:24 - cmdstanpy - INFO - Chain [1] start processing
17:03:24 - cmdstanpy - INFO - Chain [1] done processing
17:03:24 - cmdstanpy - INFO - Chain [1] start processing
17:03:24 - cmdstanpy - INFO - Chain [1]

alt.VConcatChart(...)

In [40]:
def run_dataset(dataset: pd.DataFrame):
    results = []
    for station, _df in dataset.groupby("configuration_item_id"):
        res = _df.copy()
        for col, series in _df.items():
            col = str(col)
            if col.startswith("meteo"):
                df = prepare_col(series)
                pred = fit_predict(df)
                anomalies = detect_anomalies(pred)
                res[[f'anomaly_{col}']] = anomalies[['anomaly']].replace({-1: 1, 1: 1}).values
        results.append(res)
    return pd.concat(results)

In [71]:
def compile_results(df: pd.DataFrame):
    target_cols = [x for x in df.columns if x.startswith("anomaly_")]
    df["target"] = df[target_cols].agg(lambda x: "[%s]" % ", ".join(map(str, x)), axis=1)
    return df["target"]

In [73]:
X_test = load_dataset("data/test999.csv")
results = run_dataset(X_test).set_index("id").reindex(load_dataset_by_id("data/test999.csv").index)
compile_results(results).to_csv("out/grouped/prophet-test-1-sorted.csv")

17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:12 - cmdstanpy - INFO - Chain [1] start processing
17:23:12 - cmdstanpy - INFO - Chain [1] done processing
17:23:13 - cmdstanpy - INFO - Chain [1] start processing
17:23:13 - cmdstanpy - INFO - Chain [1] done processing
17:23:13 - cmdstanpy - INFO - Chain [1] start processing
17:23:13 - cmdstanpy - INFO - Chain [1] done processing
17:23:13 - cmdstanpy - INFO - Chain [1] start processing
17:23:13 - cmdstanpy - INFO - Chain [1]